In [1]:
import whisper
import queue,threading,time
import numpy as np
from scipy.io.wavfile import write as wv
import tempfile,os
import sounddevice as sd

In [2]:
model=whisper.load_model("base.en")


In [3]:
SAMPLE_RATE=16000
CHUNK_SECONDS=1
COMMAND_ON="guard my room"
COMMAND_OFF="stop guarding"
audio_queue=queue.Queue()
guard_mode=False
stop_flag=False


In [4]:

def mic_stream():
    def callback(indata, frames, t, status):
        if status:
            pass
        audio_queue.put(indata.copy())
    with sd.InputStream(samplerate=SAMPLE_RATE, channels=1, dtype='int16', callback=callback, blocksize=int(SAMPLE_RATE*CHUNK_SECONDS)):
        while not stop_flag:
            time.sleep(0.01)


In [5]:
threading.Thread(target=mic_stream, daemon=True).start()


In [6]:
try:
    while True:
        chunk = audio_queue.get()
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
            path = tmp.name
        wv(path, SAMPLE_RATE, chunk)
        result = model.transcribe(path, language="en", fp16=False, condition_on_previous_text=False)
        text = (result.get("text") or "").strip().lower()
        os.remove(path)
        if text:
            print("Heard:", text)
            if COMMAND_ON in text and not guard_mode:
                guard_mode = True
                print("Guard mode: ON")
            elif COMMAND_OFF in text and guard_mode:
                guard_mode = False
                print("Guard mode: OFF")
except KeyboardInterrupt:
    stop_flag = True

Heard: all right,mate.
Heard: all right, please.
Heard: if you want to take some of your
Heard: okay.
Heard: okay, this is the...
Heard: this is bad.
Heard: if you can't talk.
Heard: one thing that i want.
Heard: to the west.
Heard: soon!
Heard: thank you.
Heard: thank you.


Well that  was cool i guess
now lets do vosk stuff
